<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Line_Disconnects_Device_Group_Same_Day_Prophet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [24]:
# Read a comma-separated values (csv) file into a Pandas DataFrame.
fact_table = pd.read_csv('drive/My Drive/Churn/discos_device_grp_act_vol_invol_same_day_jul.csv')

In [25]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,vol_port_basic_cdma,vol_port_basic_non_cdma,vol_port_sp_android_other,vol_port_sp_apple,vol_non_port_basic_cdma,vol_non_port_basic_non_cdma,vol_non_port_sp_android_other,vol_non_port_sp_apple,vol_port_tablet,vol_non_port_tablet
0,12/1/2016,1226,NaN,6378,6754,1511,NaN,1999,1677,112.0,5596
1,12/2/2016,1095,NaN,6999,8020,1320,NaN,2009,1825,157.0,5785
2,12/3/2016,1321,NaN,5232,5812,1667,NaN,1617,1477,120.0,5642
3,12/4/2016,1451,NaN,5274,5877,1596,NaN,1603,1503,99.0,5665
4,12/5/2016,66,NaN,432,492,53,NaN,109,94,22.0,270


In [5]:
# Split out indivudal datasets for forecasting.
# Voluntary Port Outs - Phones
vol_port_basic_cdma = fact_table[['ds', 'vol_port_basic_cdma']].rename(columns = {'vol_port_basic_cdma': 'y'})
vol_port_basic_non_cdma = fact_table[['ds', 'vol_port_basic_non_cdma']].rename(columns = {'vol_port_basic_non_cdma': 'y'})
vol_port_sp_android_other = fact_table[['ds', 'vol_port_sp_android_other']].rename(columns = {'vol_port_sp_android_other': 'y'})
vol_port_sp_apple = fact_table[['ds', 'vol_port_sp_apple']].rename(columns = {'vol_port_sp_apple': 'y'})

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma = fact_table[['ds', 'vol_non_port_basic_cdma']].rename(columns = {'vol_non_port_basic_cdma': 'y'})
vol_non_port_basic_non_cdma = fact_table[['ds', 'vol_non_port_basic_non_cdma']].rename(columns = {'vol_non_port_basic_non_cdma': 'y'})
vol_non_port_sp_android_other = fact_table[['ds', 'vol_non_port_sp_android_other']].rename(columns = {'vol_non_port_sp_android_other': 'y'})
vol_non_port_sp_apple = fact_table[['ds', 'vol_non_port_sp_apple']].rename(columns = {'vol_non_port_sp_apple': 'y'})

# Involuntary - Phones
invol_basic_cdma = fact_table[['ds', 'invol_basic_cdma']].rename(columns = {'invol_basic_cdma': 'y'})
invol_basic_non_cdma = fact_table[['ds', 'invol_basic_non_cdma']].rename(columns = {'invol_basic_non_cdma': 'y'})
invol_sp_android_other = fact_table[['ds', 'invol_sp_android_other']].rename(columns = {'invol_sp_android_other': 'y'})
invol_sp_apple = fact_table[['ds', 'invol_sp_apple']].rename(columns = {'invol_sp_apple': 'y'})

# Tablets
vol_port_tablet = fact_table[['ds', 'vol_port_tablet']].rename(columns = {'vol_port_tablet': 'y'})
vol_non_port_tablet = fact_table[['ds', 'vol_non_port_tablet']].rename(columns = {'vol_non_port_tablet': 'y'})
invol_tablet = fact_table[['ds', 'invol_tablet']].rename(columns = {'invol_tablet': 'y'})

In [6]:
# Check the individual dataset format (testing only)
vol_port_tablet.head()

,ds,y
0,12/1/2016,15.0
1,12/2/2016,38.0
2,12/3/2016,34.0
3,12/4/2016,23.0
4,12/5/2016,27.0


In [7]:
# Define the holidays
new_years_day = pd.DataFrame({
    'holiday': 'new_years_day',
    'ds': pd.to_datetime(['2017-01-01',
                          '2018-01-01',
                          '2019-01-01',
                          '2020-01-01']),
    'lower_window': 0,
    'upper_window': 0,
})

mlk_day = pd.DataFrame({
    'holiday': 'mlk_day',
    'ds': pd.to_datetime(['2017-01-16',
                          '2018-01-15',
                          '2019-01-21',
                          '2020-01-20']),
    'lower_window': 0,
    'upper_window': 0,
})

easter_sunday = pd.DataFrame({
    'holiday': 'easter_sunday',
    'ds': pd.to_datetime(['2017-04-16',
                          '2018-04-01',
                          '2019-04-21',
                          '2020-04-12']),
    'lower_window': 0,
    'upper_window': 0,
})

memorial_day = pd.DataFrame({
    'holiday': 'memorial_day',
    'ds': pd.to_datetime(['2017-05-29',
                          '2018-05-28',
                          '2019-05-27',
                          '2020-05-25']),
    'lower_window': 0,
    'upper_window': 0,
})

independence_day = pd.DataFrame({
    'holiday': 'independence_day',
    'ds': pd.to_datetime(['2017-07-04',
                          '2018-07-04',
                          '2019-07-04',
                          '2020-07-04']),
    'lower_window': 0,
    'upper_window': 0,
})

labor_day = pd.DataFrame({
    'holiday': 'labor_day',
    'ds': pd.to_datetime(['2017-09-04',
                          '2018-09-03',
                          '2019-09-02',
                          '2020-09-07']),
    'lower_window': 0,
    'upper_window': 0,
})

thanksgiving_day = pd.DataFrame({
    'holiday': 'thanksgiving_day',
    'ds': pd.to_datetime(['2017-11-23',
                          '2018-11-22',
                          '2019-11-28',
                          '2020-11-26']),
    'lower_window': 0,
    'upper_window': 0,
})

black_friday = pd.DataFrame({
    'holiday': 'black_friday',
    'ds': pd.to_datetime(['2017-11-24',
                          '2018-11-23',
                          '2019-11-29',
                          '2020-11-27']),
    'lower_window': 0,
    'upper_window': 0,
})

cyber_monday = pd.DataFrame({
    'holiday': 'cyber_monday',
    'ds': pd.to_datetime(['2017-11-27',
                          '2018-11-26',
                          '2019-12-02',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

christmas_day = pd.DataFrame({
    'holiday': 'christmas_day',
    'ds': pd.to_datetime(['2017-12-25',
                          '2018-12-25',
                          '2019-12-25',
                          '2020-12-25']),
    'lower_window': 0,
    'upper_window': 0,
})

In [8]:
holidays = pd.concat((new_years_day,
                      mlk_day,
                      easter_sunday,
                      memorial_day,
                      independence_day,
                      labor_day,
                      thanksgiving_day,
                      black_friday,
                      cyber_monday,
                      christmas_day))

In [9]:
# Define the models
# Voluntary Port Outs - Phones
vol_port_basic_cdma_model = Prophet(holidays = holidays)
vol_port_basic_non_cdma_model = Prophet(holidays = holidays)
vol_port_sp_android_other_model = Prophet(holidays = holidays)
vol_port_sp_apple_model = Prophet(holidays = holidays)

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma_model = Prophet(holidays = holidays)
vol_non_port_basic_non_cdma_model = Prophet(holidays = holidays)
vol_non_port_sp_android_other_model = Prophet(holidays = holidays)
vol_non_port_sp_apple_model = Prophet(holidays = holidays)

# Involuntary - Phones
invol_basic_cdma_model = Prophet(holidays = holidays)
invol_basic_non_cdma_model = Prophet(holidays = holidays)
invol_sp_android_other_model = Prophet(holidays = holidays)
invol_sp_apple_model = Prophet(holidays = holidays)

# Tablets
vol_port_tablet_model = Prophet(holidays = holidays)
vol_non_port_tablet_model = Prophet(holidays = holidays)
invol_tablet_model = Prophet(holidays = holidays)

In [10]:
# Fit the models
# Voluntary Port Outs - Phones
vol_port_basic_cdma_model.fit(vol_port_basic_cdma)
vol_port_basic_non_cdma_model.fit(vol_port_basic_non_cdma)
vol_port_sp_android_other_model.fit(vol_port_sp_android_other)
vol_port_sp_apple_model.fit(vol_port_sp_apple)

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma_model.fit(vol_non_port_basic_cdma)
vol_non_port_basic_non_cdma_model.fit(vol_non_port_basic_non_cdma)
vol_non_port_sp_android_other_model.fit(vol_non_port_sp_android_other)
vol_non_port_sp_apple_model.fit(vol_non_port_sp_apple)

# Involuntary - Phones
invol_basic_cdma_model.fit(invol_basic_cdma)
invol_basic_non_cdma_model.fit(invol_basic_non_cdma)
invol_sp_android_other_model.fit(invol_sp_android_other)
invol_sp_apple_model.fit(invol_sp_apple)

# Tablets
vol_port_tablet_model.fit(vol_port_tablet)
vol_non_port_tablet_model.fit(vol_non_port_tablet)
invol_tablet_model.fit(invol_tablet)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily

In [11]:
# Number of periods into the future to forecast
n = 155

In [12]:
# Create dataframes to hold predictions
# Voluntary Port Outs - Phones
vol_port_basic_cdma_future = vol_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_port_basic_non_cdma_future = vol_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_port_sp_android_other_future = vol_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_port_sp_apple_future = vol_port_sp_apple_model.make_future_dataframe(periods = n)

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma_future = vol_non_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_non_port_basic_non_cdma_future = vol_non_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_non_port_sp_android_other_future = vol_non_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_non_port_sp_apple_future = vol_non_port_sp_apple_model.make_future_dataframe(periods = n)

# Involuntary -  Phones
invol_basic_cdma_future = invol_basic_cdma_model.make_future_dataframe(periods = n)
invol_basic_non_cdma_future = invol_basic_non_cdma_model.make_future_dataframe(periods = n)
invol_sp_android_other_future = invol_sp_android_other_model.make_future_dataframe(periods = n)
invol_sp_apple_future = invol_sp_apple_model.make_future_dataframe(periods = n)

# Tablets
vol_port_tablet_future = vol_port_tablet_model.make_future_dataframe(periods = n)
vol_non_port_tablet_future = vol_non_port_tablet_model.make_future_dataframe(periods = n)
invol_tablet_future = invol_tablet_model.make_future_dataframe(periods = n)

In [13]:
# Make predictions
# Voluntary Port Outs - Phones
vol_port_basic_cdma_forecast = vol_port_basic_cdma_model.predict(vol_port_basic_cdma_future)
vol_port_basic_non_cdma_forecast = vol_port_basic_non_cdma_model.predict(vol_port_basic_non_cdma_future)
vol_port_sp_android_other_forecast = vol_port_sp_android_other_model.predict(vol_port_sp_android_other_future)
vol_port_sp_apple_forecast = vol_port_sp_apple_model.predict(vol_port_sp_apple_future)

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma_forecast = vol_non_port_basic_cdma_model.predict(vol_non_port_basic_cdma_future)
vol_non_port_basic_non_cdma_forecast = vol_non_port_basic_non_cdma_model.predict(vol_non_port_basic_non_cdma_future)
vol_non_port_sp_android_other_forecast = vol_non_port_sp_android_other_model.predict(vol_non_port_sp_android_other_future)
vol_non_port_sp_apple_forecast = vol_non_port_sp_apple_model.predict(vol_non_port_sp_apple_future)

# Involuntary - Phones
invol_basic_cdma_forecast = invol_basic_cdma_model.predict(invol_basic_cdma_future)
invol_basic_non_cdma_forecast = invol_basic_non_cdma_model.predict(invol_basic_non_cdma_future)
invol_sp_android_other_forecast = invol_sp_android_other_model.predict(invol_sp_android_other_future)
invol_sp_apple_forecast = invol_sp_apple_model.predict(invol_sp_apple_future)

# Tablets
vol_port_tablet_forecast = vol_port_tablet_model.predict(vol_port_tablet_future)
vol_non_port_tablet_forecast = vol_non_port_tablet_model.predict(vol_non_port_tablet_future)
invol_tablet_forecast = invol_tablet_model.predict(invol_tablet_future)

In [14]:
# Get the datestamp and prediction columns
# Voluntary Port Outs - Phones
vol_port_basic_cdma_forecast_subset = vol_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_CDMA'})
vol_port_basic_non_cdma_forecast_subset = vol_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_non_CDMA'})
vol_port_sp_android_other_forecast_subset = vol_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Android_Other'})
vol_port_sp_apple_forecast_subset = vol_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Apple'})

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma_forecast_subset = vol_non_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_CDMA'})
vol_non_port_basic_non_cdma_forecast_subset = vol_non_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_non_CDMA'})
vol_non_port_sp_android_other_forecast_subset = vol_non_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Android_Other'})
vol_non_port_sp_apple_forecast_subset = vol_non_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Apple'})

# Involuntary - Phones
invol_basic_cdma_forecast_subset = invol_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_CDMA'})
invol_basic_non_cdma_forecast_subset = invol_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_non_CDMA'})
invol_sp_android_other_forecast_subset = invol_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Android_Other'})
invol_sp_apple_forecast_subset = invol_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Apple'})

# Tablets
vol_port_tablet_forecast_subset = vol_port_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Tablet'})
vol_non_port_tablet_forecast_subset = vol_non_port_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Tablet'})
invol_tablet_forecast_subset = invol_tablet_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Tablet'})

In [15]:
# Check the format of the subsetted forecasts (testing only)
invol_tablet_forecast_subset.tail()

,ds,Invol_Tablet
1487,2020-12-27,172.739536
1488,2020-12-28,143.537458
1489,2020-12-29,283.401279
1490,2020-12-30,384.940597
1491,2020-12-31,411.540904


In [16]:
# Growth multiplier
gm = 1.00

In [17]:
# Replace any negative values and scale the forecast using the growth multiplier
# Voluntary Port Outs - Phones
vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'] = vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'].abs() * gm
vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'] = vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'].abs() * gm
vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'] = vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'].abs() * gm
vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'] = vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'].abs() * gm

# Voluntary non Port Outs - Phones
vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'] = vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'].abs() * gm
vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'] = vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'].abs() * gm
vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'] = vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'].abs() * gm
vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'] = vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'].abs() * gm

# Involuntary - Phones
invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'] = invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'].abs() * gm
invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'] = invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'].abs() * gm
invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'] = invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'].abs() * gm
invol_sp_apple_forecast_subset['Invol_SP_Apple'] = invol_sp_apple_forecast_subset['Invol_SP_Apple'].abs() * gm

# Tablet
vol_port_tablet_forecast_subset['Vol_Port_Tablet'] = vol_port_tablet_forecast_subset['Vol_Port_Tablet'].abs() * gm
vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'] = vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'].abs() * gm
invol_tablet_forecast_subset['Invol_Tablet'] = invol_tablet_forecast_subset['Invol_Tablet'].abs() * gm

In [18]:
# Check the format of the adjusted forecasts (testing only)
invol_tablet_forecast_subset.tail()

,ds,Invol_Tablet
1487,2020-12-27,172.739536
1488,2020-12-28,143.537458
1489,2020-12-29,283.401279
1490,2020-12-30,384.940597
1491,2020-12-31,411.540904


In [19]:
# Combine the forecasts into one table
# Voluntary Port Outs - Phones
discos_device_grp_pred_table = vol_port_basic_cdma_forecast_subset
discos_device_grp_pred_table.insert(2, 'Vol_Port_Basic_non_CDMA', vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(3, 'Vol_Port_SP_Android_Other', vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'])
discos_device_grp_pred_table.insert(4, 'Vol_Port_SP_Apple', vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'])

# Voluntary Non Port Outs - Phones
discos_device_grp_pred_table.insert(5, 'Vol_Non_Port_Basic_CDMA', vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'])
discos_device_grp_pred_table.insert(6, 'Vol_Non_Port_Basic_non_CDMA', vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(7, 'Vol_Non_Port_SP_Android_Other', vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'])
discos_device_grp_pred_table.insert(8, 'Vol_Non_Port_SP_Apple', vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'])

# Involuntary - Phones
discos_device_grp_pred_table.insert(9, 'Invol_Basic_CDMA', invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'])
discos_device_grp_pred_table.insert(10, 'Invol_Basic_non_CDMA', invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'])
discos_device_grp_pred_table.insert(11, 'Invol_SP_Android_Other', invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'])
discos_device_grp_pred_table.insert(12, 'Invol_SP_Apple', invol_sp_apple_forecast_subset['Invol_SP_Apple'])

# Tablet
discos_device_grp_pred_table.insert(13, 'Vol_Port_Tablet', vol_port_tablet_forecast_subset['Vol_Port_Tablet'])
discos_device_grp_pred_table.insert(14, 'Vol_Non_Port_Tablet', vol_non_port_tablet_forecast_subset['Vol_Non_Port_Tablet'])
discos_device_grp_pred_table.insert(15, 'Invol_Tablet', invol_tablet_forecast_subset['Invol_Tablet'])

In [20]:
# Check the format of the final predictions table (testing only)
discos_device_grp_pred_table.tail()

,ds,Vol_Port_Basic_CDMA,Vol_Port_Basic_non_CDMA,Vol_Port_SP_Android_Other,Vol_Port_SP_Apple,Vol_Non_Port_Basic_CDMA,Vol_Non_Port_Basic_non_CDMA,Vol_Non_Port_SP_Android_Other,Vol_Non_Port_SP_Apple,Invol_Basic_CDMA,Invol_Basic_non_CDMA,Invol_SP_Android_Other,Invol_SP_Apple,Vol_Port_Tablet,Vol_Non_Port_Tablet,Invol_Tablet
1487,2020-12-27,48.034002,20.168498,290.572120,637.309286,125.720902,165.734299,180.019268,645.902410,21.334926,56.787468,403.029982,204.595239,3.646391,109.783058,172.739536
1488,2020-12-28,83.786681,26.934283,434.390214,797.627707,533.940222,251.906343,931.075716,1466.652195,19.799872,48.412249,365.128316,163.096918,8.189517,1214.824136,143.537458
1489,2020-12-29,81.780514,26.809578,410.188694,756.695584,465.557993,251.704245,858.989204,1381.183358,14.048186,48.755752,553.995021,385.583570,8.488990,1052.221675,283.401279
1490,2020-12-30,84.254724,27.118438,434.124467,783.120537,420.833174,244.606519,820.980924,1316.706297,58.358946,52.863938,757.860300,552.290011,8.626438,977.039034,384.940597
1491,2020-12-31,84.450151,26.863603,438.869759,769.413573,410.632471,247.633241,813.699105,1305.464690,73.725971,64.498247,757.065933,589.537932,8.792104,964.555820,411.540904


In [21]:
# Calculate the root mean squared error (testing only)
print("Vol Port Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Basic_CDMA'] - fact_table['vol_port_basic_cdma']) ** 2)))
print("Vol Port Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Basic_non_CDMA'] - fact_table['vol_port_basic_non_cdma']) ** 2)))
print("Vol Port SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_SP_Android_Other'] - fact_table['vol_port_sp_android_other']) ** 2)))
print("Vol Port SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_SP_Apple'] - fact_table['vol_port_sp_apple']) ** 2)))
print("Vol Non Port Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Basic_CDMA'] - fact_table['vol_non_port_basic_cdma']) ** 2)))
print("Vol Non Port Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Basic_non_CDMA'] - fact_table['vol_non_port_basic_non_cdma']) ** 2)))
print("Vol Non Port SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_SP_Android_Other'] - fact_table['vol_non_port_sp_android_other']) ** 2)))
print("Vol Non Port SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_SP_Apple'] - fact_table['vol_non_port_sp_apple']) ** 2)))
print("Invol Basic CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_Basic_CDMA'] - fact_table['invol_basic_cdma']) ** 2)))
print("Invol Basic Non CDMA RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_Basic_non_CDMA'] - fact_table['invol_basic_non_cdma']) ** 2)))
print("Invol SP Android Other RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_SP_Android_Other'] - fact_table['invol_sp_android_other']) ** 2)))
print("Invol SP Apple RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_SP_Apple'] - fact_table['invol_sp_apple']) ** 2)))
print("Vol Port Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Port_Tablet'] - fact_table['vol_port_tablet']) ** 2)))
print("Vol Non Port Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Vol_Non_Port_Tablet'] - fact_table['vol_non_port_tablet']) ** 2)))
print("Invol Tablet RMSE: %f" % np.sqrt(np.mean((discos_device_grp_pred_table.loc[:1307, 'Invol_Tablet'] - fact_table['invol_tablet']) ** 2)))

Vol Port Basic CDMA RMSE: 28.065479
Vol Port Basic Non CDMA RMSE: 4.428052
Vol Port SP Android Other RMSE: 120.453123
Vol Port SP Apple RMSE: 187.281702
Vol Non Port Basic CDMA RMSE: 92.936995
Vol Non Port Basic Non CDMA RMSE: 26.346671
Vol Non Port SP Android Other RMSE: 126.974987
Vol Non Port SP Apple RMSE: 150.564388
Invol Basic CDMA RMSE: 55.337243
Invol Basic Non CDMA RMSE: 17.785309
Invol SP Android Other RMSE: 416.817734
Invol SP Apple RMSE: 460.360545
Vol Port Tablet RMSE: 5.786739
Vol Non Port Tablet RMSE: 213.040332
Invol Tablet RMSE: 317.792782


In [22]:
# Save the predictions table to a csv file
discos_device_grp_pred_table.to_csv('drive/My Drive/Churn/discos_device_grp_same_day_aug_pred_table.csv')